In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import time

In [8]:
main_url = 'https://www.jobkorea.co.kr'
browser = webdriver.Chrome('/Users/sky/Downloads/chromedriver')
job_list_jobkorea = []
url = 'https://www.jobkorea.co.kr/recruit/joblist?menucode=duty&dutyCtgr=10016#anchorGICnt_{0}'


for page_num in range(1,5):
    
    browser.get(url.format(page_num))

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    
    if soup.select('#dev-gi-list > div > div.tplList.tplJobList > table > tbody > tr') == []:
        break
    
    for tag in soup.select('#dev-gi-list > div > div.tplList.tplJobList > table > tbody > tr'):
        company = {}

        company['기업명'] = tag.select('td.tplCo > a')[0].text
        company['직무'] = tag.select('td > div > strong > a')[0].text
        
        company['경력'] = tag.select('td.tplTit > div > p.etc > span')[0].text
        company['학력'] = tag.select('td.tplTit > div > p.etc > span')[1].text
        company['근무지'] = tag.select('td.tplTit > div > p.etc > span')[2].text
        company['근무형태'] = tag.select('td.tplTit > div > p.etc > span')[3].text
    
    
        category = []
        for item in tag.select('td.tplTit > div > p.dsc'):
            category.append(item.text)
        company['분류'] = str(category)
        
        detail_url = tag.select_one('td.tplTit > div > strong > a').attrs['href']
        full_detail_url = urljoin(main_url, detail_url)
        company['상세링크'] = full_detail_url
        
        # corinfo_url = tag.select_one('td.tplCo > a').attrs['href']
        # full_detail_url = urljoin(main_url, detail_url)
        # company['기업정보'] = full_detail_url
        
        job_list_jobkorea.append(company)
           
        time.sleep(.5)
            
            
browser.close()

In [22]:
# csv로 저장
import pandas as pd
jobkorea = pd.DataFrame(job_list_jobkorea)
jobkorea.to_csv('jobkorea_recruit.csv', encoding='utf8')

사람인 및 잡코리아에서 링크를 통해 기업 정보를 가져오려 했지만 일관된 규칙을 찾지 못해서 미해결 상태 